In [3]:
from matplotlib.pyplot import *
import numpy as np
import h5py
from Data_Core.experiment import *
from Data_Core.digital_twin import *
from tqdm import tqdm
#from src.algorithms import *
#import torch
import sklearn
%load_ext autoreload
import scipy

In [113]:
import numpy as np
import torch
from tqdm import tqdm
from functools import partial

def initialize(X, num_clusters, seed):
    """
    Initialize cluster centers
    """

    num_samples = len(X)
    if seed == None:
        indices = np.random.choice(num_samples, num_clusters, replace = False)
    else:
        np.random.seed(seed)
        indices = np.random.choice(num_samples, num_clusters, replace = False)
    initial_state = X[indices]
    return initial_state


def kmeans_t(
        X, 
        num_clusters,
        distance = 'euclidean',
        converg='cross_entropy',
        cluster_centers = [],
        tol = 1e-4,
        tqdm_flag = True,
        iter_limit = 20,
        device = torch.device('cpu'),
        seed = None
):
    """
    perform kmeans
    """
    if tqdm_flag:
        print(f'Running Kmeans on {device}..')
    
    if distance == 'euclidean':
        pairwise_distance_function = partial(pairwise_distance, device = device, tqdm_flag = tqdm_flag)
    elif distance == 'iou':
        pairwise_distance_function = partial(pairwise_iou, device = device, tqdm_flag = tqdm_flag)
    elif distance == 'cross_entropy':
        pairwise_distance_function = partial(cross_entropy_loss, device = device, tqdm_flag = tqdm_flag)
    else:
        raise NotImplementedError

    # Convert to Float
    X = X.float()

    # Transfer to Device
    X = X.to(device)

    if type(cluster_centers) == list:
        initial_state = initialize(X, num_clusters, seed = seed)
    else:
        if tqdm_flag:
            print('Resuming')

        # Find closest point to initial cluster center
        initial_state = cluster_centers

        print('X',X)
        print('initial state',initial_state)

        
        dis = pairwise_distance_function(X, initial_state)

        choice_points = torch.argmin(dis, dim = 0)
        initial_state = X[choice_points]
        initial_state = initial_state.to(device)

    iteration = 0
    if tqdm_flag:
        tqdm_meter = tqdm(desc = '[Running Kmeans]')
    while True:

        dis = pairwise_distance_function(X, initial_state)
        choice_clusters = torch.argmin(dis, dim = 1)
        initial_state_pre = initial_state.clone()

        for index in range(num_clusters):
            selected = torch.nonzero(choice_clusters == index).squeeze().to(device)
            selected = torch.index_select(X, 0, selected)

            if selected.shape[0] == 0:
                selected = X[torch.randint(len(X), (1,))]

            initial_state[index] = selected.mean(dim = 0)
        
        center_shift = 0
        center_intersect = 0

        if converg == 'euclidean':
            center_shift = torch.sum(
            torch.sqrt(
                torch.sum((initial_state - initial_state_pre)**2, dim = 1)
            ))
        if converg=='cross_entropy':
            center_shift=cross_entropy_loss(initial_state,initial_state_pre)

            
            
        # if distance == 'iou':
        #     inter = torch.sum((initial_state.int() & initial_state_pre.int()), dim = 1)
        #     union = torch.sum((initial_state.int() | initial_state_pre.int()), dim = 1)

        #     center_intersect = 1 - inter/union

        # Increment iteration
        iteration += 1
        inertia = 0

        # Update tqdm meter
        if tqdm_flag:
            tqdm_meter.set_postfix(
                iteration = f'{iteration}',
                center_shift = f'{center_shift**2:0.6f}',
                tol = f'{tol:0.6f}'
            )
            tqdm_meter.update()

        if distance == 'euclidean' and center_shift ** 2 < tol:
            inertia = 0
            for index in range(num_clusters):
                selected = torch.nonzero(choice_clusters == index).squeeze().to(device)
                selected = torch.index_select(X, 0, selected)
                inertia += torch.sum(((selected - initial_state[index]) ** 2))
            break
        # if distance == 'iou' and center_intersect < tol:
        #     break
        if iter_limit != 0 and iteration >= iter_limit:
            break

    return choice_clusters.cpu(), initial_state.cpu(), inertia


def kmeans_predict(
        X,
        cluster_center,
        distance = 'euclidean',
        device = torch.device('cpu'),
        tqdm_flag = True
):
    # Predict using cluster centers

    if tqdm_flag:
        print(f'Predicting on {device}')

    if distance == 'euclidean':
        pairwise_distance_function = partial(pairwise_distance, device = device, tqdm_flag = tqdm_flag)
    elif distance == 'iou':
        pairwise_distance_function = partial(pairwise_iou, device = device, tqdm_flag = tqdm_flag)
    elif distance == 'cross_entropy':
        pairwise_distance_function = partial(cross_entropy_loss, device = device, tqdm_flag = tqdm_flag)
    else:
        raise NotImplementedError
    
    # Convert to Float
    X = X.float()

    # Transfer to Device
    X = X.to(device)

    dis = pairwise_distance_function(X, cluster_center)
    choice_clusters = torch.argmin(dis, dim = 1)

    return choice_clusters.cpu()


def pairwise_distance(data1, data2, device = torch.device('cpu'), tqdm_flag = True):
    # if tqdm_flag:
    #     print(f'device is :{device}')
    
    # transfer to device
    data1, data2 = data1.to(device), data2.to(device)

    # N*1*M
    A = data1.unsqueeze(dim = 1)

    # 1*N*M
    B = data2.unsqueeze(dim = 0)

    dis = (A - B) ** 2.0
    # return N*N matrix for pairwise distance
    dis = dis.sum(dim = -1).squeeze()
    return dis

def pairwise_iou(data1, data2, device = torch.device('cpu'), tqdm_flag = True):
    # if tqdm_flag:
    #     print(f'device is :{device}')
    
    # transfer to device
    data1, data2 = data1.to(device), data2.to(device)

    A = data1.int()
    B = data2.int()

        # N*1*M
    A = A.unsqueeze(dim = 1)

    # 1*N*M
    B = B.unsqueeze(dim = 0)

    inter = torch.sum((A & B), dim = 2)
    inter2 = torch.sum((torch.logical_not(A) & torch.logical_not(B)), dim = 2)
    union = torch.sum((A | B), dim = 2)
    union2 = torch.sum((torch.logical_not(A) | torch.logical_not(B)), dim = 2)

    #inter3 = torch.sum((torch.logical_not(A) & B), dim = 2)
    #union3 = torch.sum((torch.logical_not(A) | B), dim = 2)
    
    dis = 1 - (inter/union )#* inter2/union2))
    # return N*N matrix for pairwise distance
    dis = dis.squeeze()
    return dis
def cross_entropy_loss(data1,data2,device = torch.device('cpu'), tqdm_flag = True):
    data1, data2 = data1.to(device), data2.to(device)
    import torch.nn.functional as F
    import torch
    softmax = F.softmax(data1.float(), dim=0)
    data1=softmax
    loss = 0
 
    # Doing cross entropy Loss
    for i in range(len(data1)):
        # Here, the loss is computed using the
        # above mathematical formulation.
        loss = loss + (-1 * np.dot(data2[i],np.log(data1[i])))

    return loss

In [4]:
find_index = lambda wavelenghts, w : np.argmin(np.abs(wavelenghts - w))

In [5]:
def Raman_data_loader(filename):

    with h5py.File(filename, 'a') as output_file:
         
        properties = output_file['properties']
        
        exp_properties = {'step_size' : np.array(properties['step_size'])[0],
                          'speed' : np.array(properties['speed']),
                          'n_points' : np.array(properties['n_points'])
            }
        
        wavelengths = np.array(output_file['properties']['x_data'])
        
        
        spot_numbers = [int(s.split('_')[-1]) for s in list(output_file['data'].keys()) if 'spot' in s ]
        
        Nx,Ny = output_file['properties']['n_points'][0], output_file['properties']['n_points'][1]
        Nl = len(wavelengths)
        spectral_signal = np.zeros([Nx,Ny,Nl])
        
        
        for _i, spot_number in enumerate(spot_numbers):

            ix, iy = int(spot_number//Ny), int(spot_number%Ny)
            
            spot = 'spot_'+str(spot_number)
            data = np.array(output_file['data'][spot]['raw_data'])
                      
            spectral_signal[ix,iy,:] = data
            

    return spectral_signal, wavelengths, exp_properties

In [6]:
filename = "2023913_1110.h5"
#filename = "2023811_1451.h5"
spectrum_raman, wavelengths_raman, exp_properties = Raman_data_loader(filename)

In [7]:
from scipy import sparse
from scipy.sparse.linalg import spsolve

def baseline_als_optimized(y, lam, p, niter = 10):
    L = len(y)
    D = sparse.diags([1, -2, 1],[0, -1, -2], shape = (L, L - 2))
    D = lam * D.dot(D.transpose()) # Precompute this term since it does not depend on `w`
    w = np.ones(L)
    W = sparse.spdiags(w, 0, L, L)
    for i in range(niter):
        W.setdiag(w) # Do not create a new matrix, just update diagonal values
        Z = W + D
        z = spsolve(Z, w*y)
        w = p * (y > z) + (1 - p) * (y < z)
    return z

In [8]:
lam = 1e2
p = 1e-1
    
pro_data = 1*spectrum_raman
for i in tqdm(range(0, pro_data.shape[0])):
    for j in range(0, pro_data.shape[1]):
        # print(i,j,end='\r')
        spec = 1 * pro_data[i, j, :]
        pro_data[i, j, :] = spec - baseline_als_optimized(spec, lam = lam, p = p, niter = 10)

100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [07:26<00:00,  1.79s/it]


In [9]:
pro_data_norm=pro_data.copy()

In [10]:
offset_l = 31
offset_m = 328

mask = pro_data_norm[:, :, offset_l:offset_m].reshape(pro_data.shape[0]*pro_data.shape[1], -1)

In [11]:
threshold = 1.27e-9

mask_offset = np.array(mask)
mask_t = np.array([(mask_offset[:, i])*((mask_offset[:, i] > threshold)) for i in range(mask_offset.shape[-1])])

In [12]:
def read_ref(mineral):
    df = pd.read_csv(mineral + '_raman.txt',header=13,names=['wavenumber','int'],skipfooter=4,engine='python' )
    return np.array(df['wavenumber']),np.array(df['int'])

In [14]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(mask_t)
mask_min_max=scaler.transform(mask_t)



In [15]:
entropy_masks=mask_min_max*255

In [16]:
def entropy_calc(map_gr):
    import cv2
    from scipy.stats import entropy

    #image

    _bins = 200

    hist, _ = np.histogram(map_gr, bins=_bins, range=(0, _bins))

    prob_dist = hist / hist.sum()
    image_entropy = entropy(prob_dist, base=2)
    return image_entropy

In [17]:
offset_l = 31
offset_m = 328
wavelengths_raman.shape
wavelengths=wavelengths_raman[offset_l:offset_m]

minerals = {
    
    'Albite':[508.1],
    'Background':[733.1],
    'Petalite':[491.3],
    'Quartz':[463.92],
    'Spodumene':[704.5]}

mineral_list = list(minerals.keys())

find_index = lambda wavelenghts,w : np.argmin(np.abs(wavelengths-w))

significant_w=[]
for i,mineral in enumerate(mineral_list):
    wn=find_index(wavelengths,minerals[mineral][0])
    significant_w.append(wn)


image_entropy=[]
for i in range(297):
    map_gr=entropy_masks[i]
    image_entropy.append(entropy_calc(map_gr))




In [18]:
relevant_entropies=[]
for i in significant_w:
    relevant_entropies.append(image_entropy[i])
    
max_en=np.max(relevant_entropies)+0.2
min_en=np.min(relevant_entropies)-0.2
#min_en=5.2
#max_en=6.3

new_maps_cut=[]
for i in range(entropy_masks.shape[0]):
    if min_en<=image_entropy[i]<=max_en:
        new_maps_cut.append(entropy_masks[i,:])
        
new_maps_cut=np.array(new_maps_cut)


In [23]:
mask_torch = torch.from_numpy(new_maps_cut)

In [27]:
mask_torch.shape

torch.Size([275, 25000])

In [114]:
kmeans_cpu = kmeans_t(X = mask_torch, num_clusters = 5, distance = 'euclidean',converg='cross_entropy')

Running Kmeans on cpu..


[Running Kmeans]: 0it [00:00, ?it/s]C:\Users\Joana\AppData\Local\Temp\ipykernel_20160\1709818985.py:229: RuntimeWarning: divide by zero encountered in log
  loss = loss + (-1 * np.dot(data2[i],np.log(data1[i])))
[Running Kmeans]: 20it [00:03,  6.56it/s, center_shift=nan, iteration=20, tol=0.000100]


In [128]:
def cross_entropy_loss(data1,data2,device = torch.device('cpu'), tqdm_flag = True):
    data1, data2 = data1.to(device), data2.to(device)
    import torch.nn.functional as F
    import torch
    softmax = F.softmax(data1.float(), dim=0)
    data1=softmax
    loss = 0
 
    # Doing cross entropy Loss
    for i in range(len(data1)):
        #print(f"i: {i}, data1 size: {data1.size()}, data2 size: {data2.size()}")
        # Here, the loss is computed using the
        # above mathematical formulation.
        print('data1 of the i ',data1[i])
        print('loss i', loss)
        print('data 2', data2[i])
        print('data1', data1[i])
        print('softmax', softmax[i])
        print('log',np.log(data1[i]))
        print('dot',np.dot(data2[i],np.log(data1[i])))
        
        
        loss = loss + (-1 * np.dot(data2[i],np.log(data1[i])))
        if i>0:
            break

    return loss

data1=torch.from_numpy(np.array([[0,2,3],
                [1,2,3]]))
data2=torch.from_numpy(np.array(np.array([[4,5,6],
                [4,5,6]])))

In [130]:
cross_entropy_loss(mask_torch[0],mask_torch[13])
data1.size()

data1 of the i  tensor(0.)
loss i 0
data 2 tensor(14.1885, dtype=torch.float64)
data1 tensor(0.)
softmax tensor(0.)
log tensor(-inf)
dot -inf
data1 of the i  tensor(0.)
loss i inf
data 2 tensor(0.3973, dtype=torch.float64)
data1 tensor(0.)
softmax tensor(0.)
log tensor(-inf)
dot -inf


C:\Users\Joana\AppData\Local\Temp\ipykernel_20160\972483849.py:19: RuntimeWarning: divide by zero encountered in log
  print('log',np.log(data1[i]))
C:\Users\Joana\AppData\Local\Temp\ipykernel_20160\972483849.py:20: RuntimeWarning: divide by zero encountered in log
  print('dot',np.dot(data2[i],np.log(data1[i])))
C:\Users\Joana\AppData\Local\Temp\ipykernel_20160\972483849.py:23: RuntimeWarning: divide by zero encountered in log
  loss = loss + (-1 * np.dot(data2[i],np.log(data1[i])))


torch.Size([2, 3])

In [105]:
data1=np.array([[0,0.2,0.3],
                [1,2,3]])

import torch.nn.functional as F
import torch
softmax = F.softmax(mask_torch[0], dim=0)
softmax

tensor([1.3823e-112, 1.3823e-112, 7.6490e-109,  ..., 1.3418e-108,
        1.3823e-112,  9.0682e-94], dtype=torch.float64)

In [107]:
log(1.3832e-112)

-257.56513075994616